In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [85]:
def draw_registration_result(source, target, transformation=None):
    if transformation is None:
        transformation = np.asarray([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def execute_global_registration(
        source_down,
        target_down,
        source_fpfh,
        target_fpfh,
        distance_threshold
):
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source=source_down,
        target=target_down,
        source_feature=source_fpfh,
        target_feature=target_fpfh,
        mutual_filter=False,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        # estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(200000, 0.99999)
    )
    return result


def preprocess_point_cloud(pcd, voxel_size=0):
    # pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down = pcd

    radius_ = radius(pcd)
    radius_normal = radius_ * 3
    radius_feature = radius_ * 20
    max_nn = 200
    # radius_feature = voxel_size * 5
    # radius_normal = voxel_size * 2

    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn))
    pcd_down.estimate_normals()
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn))
    return pcd_down, pcd_fpfh


def show_range_coords(ply):
    ply_np = np.asarray(ply.points)
    print()
    for i in range(3):
        print(
            ply_np[:, i].min(),
            ply_np[:, i].max(),
            "range:",
            ply_np[:, i].max() - ply_np[:, i].min()
        )

def radius(ply):
    distances = ply.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    print("radius:", radius)
    return radius

In [86]:
# ply_source = "D:/sfm_dense/street_1_0_real_time/target.ply"
# ply_source = "D:/sfm_dense/street_2_0/dense/0/color_pd30_8bit_sample.ply"

# ply_target = "D:/sfm_dense/street_1_0_query/dense/0/fused.ply"
# ply_target = "D:/sfm_dense/street_1_0_query/query.sparse.ply"
# ply_target ="E:/University/Thesis/color_pd30_8bit_small_filtered.ply"

# street_4_0
# ply_source = "D:/sfm_dense/street_4_0_1/dense/0/fused_scaled_uniformed.ply"
# ply_target = "D:/sfm_dense/color_pd30_8bit_street_4_0_sliced_uniformed.ply"

# street_4_3
ply_source = "D:/sfm_dense/street_4_3_3/dense/0/fused_uniformed_aligned.ply"
ply_target = "D:\sfm_dense\street_4_3_3\color_pd30_8bit_small_street_4_3_3.ply"


source = o3d.io.read_point_cloud(ply_source)
target = o3d.io.read_point_cloud(ply_target)

# Preprocessing source
print("source")
scale = 13
show_range_coords(source)
source = source.voxel_down_sample(voxel_size=radius(source)*1.5)

source = source.select_by_index(np.where(np.asarray(source.points)[:, 2] < -.3)[0])
source.scale(scale, center=source.get_center())
# o3d.visualization.draw_geometries([source, target])


# Preprocessing target
print("target")
show_range_coords(target)
target = target.select_by_index(np.where(np.asarray(target.points)[:, 2] < -3)[0])



draw_registration_result(source, target)

source_down, source_fpfh = preprocess_point_cloud(source)
target_down, target_fpfh = preprocess_point_cloud(target)


source

-5.18475866317749 5.746166229248047 range: 10.930924892425537
-6.182560920715332 3.956705331802368 range: 10.1392662525177
-0.5147021412849426 1.6505919694900513 range: 2.165294110774994
radius: 0.05778257459745971
target

-81.13799711305182 85.67450288694818 range: 166.8125
-76.10136783309281 68.89863216690719 range: 145.0
-11.236785264176717 16.793217329817423 range: 28.03000259399414
radius: 2.630230003760081
radius: 2.4452413620393094


In [87]:
print("Initial alignment")
trans_init = np.asarray([
    [0.862, 0.011, -0.507, 110.9],
    [-0.139, 0.967, -0.215, 220.1],
    [0.487, 0.255, 0.835, -1.4],
    [0.0, 0.0, 0.0, 1.0]
])
draw_registration_result(source, target, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, radius(source), trans_init)
print(evaluation)
print()

Initial alignment
radius: 2.630230003760081
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.



In [88]:
source_down = source_down.translate([100, 100, 0], relative=False)

result_fast = execute_global_registration(
    source_down,
    target_down,
    source_fpfh,
    target_fpfh,
    radius(source)*1.5
)
print("Global", result_fast)
draw_registration_result(source_down, target_down, result_fast.transformation)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, result_fast.transformation)
print("Global", evaluation)
print("Global Transformation",)
print(result_fast.transformation)
print()

radius: 2.6302300037600803
Global RegistrationResult with fitness=9.427208e-01, inlier_rmse=1.316531e+00, and correspondence_set size of 1185
Access transformation to get result.
Global RegistrationResult with fitness=6.483691e-01, inlier_rmse=5.532200e-01, and correspondence_set size of 815
Access transformation to get result.
Global Transformation
[[ 5.38734591e-01 -8.42445905e-01  7.06674185e-03  2.53697368e+01]
 [ 8.42122165e-01  5.38246747e-01 -3.34768558e-02 -1.48726709e+02]
 [ 2.43987892e-02  2.39862001e-02  9.99414509e-01 -1.29135057e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [89]:
threshold = 1
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source_down,
    target_down,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=6.626889e-01, inlier_rmse=5.223891e-01, and correspondence_set size of 833
Access transformation to get result.
Transformation is:
[[ 5.44446215e-01 -8.38774711e-01  5.94160171e-03  2.47639086e+01]
 [ 8.38505340e-01  5.44057675e-01 -3.01668849e-02 -1.49167377e+02]
 [ 2.20706462e-02  2.14063111e-02  9.99527216e-01 -1.26211217e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [90]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source,
    target,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=6.603023e-01, inlier_rmse=5.096639e-01, and correspondence_set size of 830
Access transformation to get result.
Transformation is:
[[ 6.60372333e-01 -7.50935691e-01  1.99245699e-03  3.08888396e+00]
 [ 7.50631980e-01  6.60024821e-01 -3.03127997e-02 -1.54736644e+02]
 [ 2.14478921e-02  2.15133362e-02  9.99538476e-01 -1.25494599e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

